<a href="https://colab.research.google.com/github/Meaveryway/Backend_AntTech/blob/master/2_Indexing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
!pip install langdetect
!pip install tika
!pip install pdfplumber

from langdetect import detect
import time
from datetime import datetime
import csv
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os.path
import re

import pdfplumber
from tika import parser
from bs4 import BeautifulSoup
from io import StringIO
from tika import detector


from time import sleep
from random import randint
import dateutil.parser as dateParser
import concurrent.futures

     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 5.6MB 22.8MB/s 
     |████████████████████████████████| 143kB 50.2MB/s 
     |████████████████████████████████| 1.9MB 43.6MB/s 
  Created wheel for pdfplumber: filename=pdfplumber-0.5.28-cp37-none-any.whl size=32222 sha256=6a40103c7e0205630ce631d2363ad43a755037f32f04bfa63a0eb19333cbe2ed
  Stored in directory: /root/.cache/pip/wheels/5a/9b/3d/37e37d93650be964fdd2a9babf174214fec8a0f4d222d474ab
Successfully built pdfplumber


In [2]:
!pip freeze > requirements.txt

In [3]:
!pip install elastic_enterprise_search==7.10.0a1
from elastic_enterprise_search import AppSearch

  Created wheel for tika: filename=tika-1.24-cp37-none-any.whl size=32885 sha256=7f9d173cfc66f4901ed37bb5bc308cac697be58b777e8fa505f67017bc0ea369
  Stored in directory: /root/.cache/pip/wheels/73/9c/f5/0b1b738442fc2a2862bef95b908b374f8e80215550fb2a8975
Successfully built tika


In [4]:
#context = ssl._create_unverified_context()
import requests
requests.packages.urllib3.disable_warnings()

import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context

In [5]:
# Dataframes en mode interactif
%load_ext google.colab.data_table
#%unload_ext google.colab.data_table

# Access To Google Drive
In order to write files on there for persistency.

In [6]:
DRIVE_PATH = "/content/drive/MyDrive/Théses-Algérie/Data"
MISSED_LINKS_PATH = DRIVE_PATH + "missingno"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# File downloading

In [ ]:
def download_file(download_url, filename):
    r = requests.get(download_url, stream = True) 
    with open(filename,"wb") as pdf: 
      for chunk in r.iter_content(chunk_size=1024): 
        # One chunk at a time
        if chunk: 
          pdf.write(chunk) 

# Connecting to the Engine 

In [7]:
# Connecting to an instance on VPS w/ an App Search private key
app_search = AppSearch(
    "https://www.theses-dz.ml:8080",
    http_auth="private-vq7y3ap5tkax1r7wouo2xzfm",
    ca_certs=False,
    verify_certs=False,
)
engine_name = "theses-dz"
engine_name_full_text = "full-text-sampler"
app_search.get_schema(engine_name)

/usr/local/lib/python3.7/dist-packages/elastic_transport/connection/http_urllib3.py:183: UserWarning: Connecting to 'https://www.theses-dz.ml:8080' using SSL with verify_certs=False is insecure
  % self.base_url


{'keywords': 'text', 'additional_urls': 'text', 'body_content': 'text', 'domains': 'text', 'language': 'text', 'abstract': 'text', 'title': 'text', 'type': 'text', 'body': 'text', 'meta_keywords': 'text', 'url': 'text', 'meta_description': 'text', 'field': 'text', 'publication_date': 'number', 'publisher': 'text', 'links': 'text', 'identifier_uri': 'text', 'authors': 'text'}

## <font color="cyan"> **File curently being indexed** </font>
### <font color="red"> **watch out for doc type, and file names** </font>

In [ ]:
FILE_NAME = "MSILA_" + "SOCIALES"
df_details_to_index = pd.read_csv(DRIVE_PATH + '/' + FILE_NAME + '_DETAILS')
#TYPE_NAME = "Thèse de Doctorat"
#TYPE_NAME = "Mémoire de Magister"
#TYPE_NAME = "Mémoire de Master"
TYPE_NAME = "idle"


In [ ]:
print(FILE_NAME)
print(df_details_to_index.shape)

MSILA_SOCIALES
(3718, 14)


In [ ]:
launch_indexer()

In [24]:
def launch_indexer():
  part_to_iter = df_details_to_index
  total_size = df_details_to_index.shape[0]
  count = 0
  for index, item in part_to_iter.iterrows():

    id = item["id"]
    title = item['title']
    authors = item['author']
    field= item['field']
    abstract = item['abstract']
    publication_date = item['date_issued']
    language = item['language']
    keywords = item['keywords']
    print(title)

    doc_type = TYPE_NAME if TYPE_NAME != "idle" else item['type']
    institute = item['institute_name']
    pdf_path = item['file_url']
    pdf_path = pdf_path.replace("jspui//jspui", "jspui")
    identifier_uri = item['identifier_uri']
    authors = re.findall(r"\'(.*?)\'", authors)
    keywords = re.findall(r"\$(.*?)\$", keywords)
    if ("" in keywords):
      keywords.remove("")
      
    #download_file(pdf_path, str(id))
    #print("file downloaded")
    #rawText = parser.from_file('/content/' + str(id))
    #print("file parsed")

    app_search.index_documents(
        engine_name=engine_name,
        body=[{
              "id": id,
              "title": title,
              "abstract": abstract,
              "authors": authors,
              "publication_date": publication_date,
              "field": field,
              "keywords": keywords,
              "type": doc_type,
              "publisher": institute,
              "url": pdf_path,
              "identifier_uri": identifier_uri,
              "language": language,
              #"body": ""
        }]
    )
    print("------------------------ {}/{}".format(count, total_size))
    count = count + 1

## Loop-Through

In [ ]:
def repo_file_lister(university):
  filenames= []
  for filename in os.listdir(DRIVE_PATH):
    if university.lower() in filename.lower() and 'detail' in filename.lower():
      filenames.append(filename)
  return filenames
file_names = repo_file_lister('MSILA')

['MSILA_TECHNOLOGIE_DETAILS',
 'MSILA_SCIENCES_DETAILS',
 'MSILA_URBA_DETAILS',
 'MSILA_DROIT_DETAILS',
 'MSILA_SPORT_DETAILS',
 'MSILA_ECONOMIE_DETAILS',
 'MSILA_LETTRES_DETAILS',
 'MSILA_SOCIALES_DETAILS']

In [25]:
for file_name in file_names:
  FILE_NAME = file_name
  df_details_to_index = pd.read_csv(DRIVE_PATH + '/' + FILE_NAME)
  #TYPE_NAME = "Thèse de Doctorat"
  #TYPE_NAME = "Mémoire de Magister"
  #TYPE_NAME = "Mémoire de Master"
  TYPE_NAME = "idle"
  print(FILE_NAME)
  print(df_details_to_index.shape)
  launch_indexer()

NameError: ignored

---
---
---

# Single indexation (manual)

In [ ]:
TITLE = ''
ID = int(str(int(hashlib.md5(str(TITLE).encode('utf-8')).hexdigest(), 16))[:16])
LINK = ''
PUBLISHER = ''
ABSTRACT = ""

AUTHORS = ['', '']
keywords = ['', '']

YEAR = 0
LANGUAGE = ''

FIELD = ''
TYPE = ''

FILE_URL = ""

#------------------------------------------
file_to_index={
          "id": ID,
          "title": TITLE,
          "abstract": ABSTRACT,
          "authors": AUTHORS,
          "publication_date": YEAR,
          "field": FIELD,
          "keywords": KEYWORDS,
          "type": TYPE,
          "publisher": PUBLISHER,
          "url": FILE_URL,
          "identifier_uri": LINK,
          "language": LANGUAGE,
          #"body": ""
      }]

In [ ]:
app_search.index_documents(engine_name, file_to_index)

---
---
---
###### Some patchwork

In [ ]:
df_details = pd.read_csv(DRIVE_PATH + '/' + "OUMELBOUAGHI_MASTER" + '_DETAILS')
df_details['field'] = df_details['field'].str.replace("قسم الحقوق", "Droit")
df_details['field'] = df_details['field'].str.replace("قسم اللغة و الأدب العربي", "Langue Et Littérature Arabe")
df_details['field'] = df_details['field'].str.replace("قسم العلوم الاقتصادية", "Économie")
df_details['field'] = df_details['field'].str.replace("قسم الكيمياء", "Chimie")
df_details['field'] = df_details['field'].str.replace("قسم الرياضيات", "Mathématiques")
df_details['field'] = df_details['field'].str.replace("قسم اللغة الفرنسية", "Langue Et Littérature Française")
df_details['field'] = df_details['field'].str.replace("قسم الفيزياء", "Physique")
df_details['field'] = df_details['field'].str.replace("قسم العلوم المالية والتسيير", "Finance Et Comptabilité")
df_details['field'] = df_details['field'].str.replace("قسم العلوم المالية والمحاسبة", "Finance Et Comptabilité")
df_details['field'] = df_details['field'].str.replace("قسم علوم الطبيعة والحياة", "Sciences De La Nature Et De La Vie")
df_details['field'] = df_details['field'].str.replace("قسم العلوم الإنسانية", "Sciences Humaines")
df_details['field'] = df_details['field'].str.replace("قسم علوم المادة", "Sciences De La Matière")
df_details['field'] = df_details['field'].str.replace("قسم الإعلام الالي", "Informatique")
df_details['field'] = df_details['field'].str.replace("قسم التربية البدنية والرياضية", "Sciences Et Techniques Des Activités Physiques Et Sportives")
df_details['field'] = df_details['field'].str.replace("قسم الهندسة الكهربائية", "Génie Eléctrique Et Electronique")
df_details['field'] = df_details['field'].str.replace("قسم العلوم الإجتماعية", "Sciences Sociales")
df_details['field'] = df_details['field'].str.replace("قسم علوم التسيير", "Gestion")
df_details['field'] = df_details['field'].str.replace("قسم تسيير التقنيات الحضرية", "Urbanisme")
df_details['field'] = df_details['field'].str.replace("قسم الجغرافيا وتهيئة الإقليم", "Géographie Et Aménagement Du Territoire")
df_details['field'] = df_details['field'].str.replace("قسم اللغة الإنجليزية", "Langue et Littérature Anglaise")
df_details['field'] = df_details['field'].str.replace("قسم الهندسة المعمارية", "Architecture")
df_details['field'] = df_details['field'].str.replace("قسم العلوم التجارية", "Sciences Commerciales")
df_details['field'] = df_details['field'].str.replace("قسم الهندسة المدنية", "Génie Civil")
df_details['field'] = df_details['field'].str.replace("قسم الري", "Hydraulique")
df_details['field'] = df_details['field'].str.replace("قسم هندسة الطرائق", "Génie Des Procédés")
df_details['field'] = df_details['field'].str.replace("قسم الهندسة الميكانيكية", "Génie Mécanique")
df_details['field'] = df_details['field'].str.replace("قسم الجيولوجيا", "Mines Et Géologie")
df_details['field'] = df_details['field'].str.replace("قسم العلوم السياسية", "Sciences Politiques")


In [ ]:
df_details_field = df_details['field']
for detail in df_details_field:
  if "قسم" in detail:
    print(detail)

قسم القياسات الفيزيائية


In [ ]:
df_details.head(5)

In [ ]:
#df_details.to_csv(DRIVE_PATH + '/' + "OUMELBOUAGHI_MASTER" + '_DETAILS')

## Files volume estimation

In [ ]:
filenames= []
total_df = []
for filename in os.listdir(DRIVE_PATH):
  if 'detail' in filename.lower():
    filenames.append(filename)

In [ ]:
for filename in filenames:
  print(filename)
  df = pd.read_csv(DRIVE_PATH + '/' + filename)
  df_files = pd.DataFrame(None, columns=['og', 'size', 'multiplicator'])
  df_files['og'] = df['file_size']
  df_files['size'] = df_files['og'].apply(lambda x: str(x))
  df_files['size'] = df_files['size'].apply(lambda x: float(re.sub('[^\d.]','', x.lower().replace('idle', '0').replace(',', '.')))) #float after removing all non numeric (except .) after converting , to . and idle to 0 (no file size in some repositories)
  df_files['multiplicator'] = df_files['og'].apply(lambda x: 1 if 'mb' in x.lower() else 0.001 if 'kb' in x.lower() else 0)
  df_files['size'] = df_files['size'] * df_files['multiplicator']
  total_size = df_files.sum(axis = 0)

  total_df.append([filename, total_size['size']])

In [ ]:
df_to_write = pd.DataFrame(total_df, columns=['repository', 'volume'])
df_to_write.to_csv('/content/drive/MyDrive/Théses-Algérie/file_sizes.csv', index = False)

In [ ]:
total_df = pd.read_csv('/content/drive/MyDrive/Théses-Algérie/file_sizes.csv')
total_size = total_df.sum(axis=0)['volume']

excluded_repositories = total_df.loc[total_df['volume'] < 1].shape[0]
total_repositories = total_df.shape[0] - excluded_repositories

print("Logged a total volume of {}Gb in {} sub_repositories (with {} excluded sub_repositories)".format(int(total_size/1000), total_repositories, excluded_repositories))

Logged a total volume of 569Gb in 75 sub_repositories (with 24 excluded sub_repositories)


# Full-text sample building

In [94]:
def digitize_file_pdfplumber(file_path):
  text = ""
  with pdfplumber.open(file_path) as pdf:
    pages = pdf.pages
    for page in pages:
      text = text + page.extract_text()
  pdf.close()
  return(text)
  

def digitize_file_tika(file_path):
    file_extension = detector.from_file(file_path)
    if 'pdf'in str(file_extension).lower():
      print("Parsing PDF")
      file_data = []
      buffer = StringIO()
      data = parser.from_file(file_path, xmlContent=True)
      xhtml_data = BeautifulSoup(data['content'], features="lxml")
      for page, content in enumerate(xhtml_data.find_all('div', attrs={'class': 'page'})):
          #print('Parsing page {} of pdf file...'.format(page+1))
          buffer.write(str(content))
          parsed_content = parser.from_buffer(buffer.getvalue())
          buffer.truncate()
          file_data.append({'id': str(page+1), 'content': parsed_content['content'].replace('\n\n', '').replace('\n', ' ')})
      #Clean-up
      for i in reversed(range(1,len(file_data))) :
        content = file_data[i]['content']
        substract = file_data[i-1]['content']
        modified_content = content.replace(substract, "")
        file_data[i]['content'] = modified_content
        file_data[i] = file_data[i]['content']
      return file_data
    
    else:
      text = data = parser.from_file(file_path)['content']
      return(text)

def digitize_file(file_path):
  file_extension = detector.from_file(file_path)
  if 'pdf'in str(file_extension).lower():
    print("going for Plumber")
    fulltext = digitize_file_pdfplumber(file_path)
  else:
    fulltext = digitize_file_tika(file_path)
    print("going for Tika")
  return fulltext

def download_file(download_url, filename):
    r = requests.get(download_url, stream = True) 
    with open(filename,"wb") as pdf: 
      for chunk in r.iter_content(chunk_size=1024): 
        # One chunk at a time
        if chunk: 
          pdf.write(chunk) 

In [109]:
def index_fulltext(item):
    id = item["id"]['raw']

    #check if file already indexed

    title = item['title']['raw']
    authors = item['authors']['raw']
    field= item['field']['raw']
    abstract = item['abstract']['raw']
    publication_date = item['publication_date']['raw']
    language = item['language']['raw']
    keywords = item['keywords']['raw']
    identifier_uri = item['identifier_uri']['raw']
    publisher = item['publisher']['raw']
    doc_type = item['type']['raw']
    url = item['url']['raw']
    print(title)

      
    download_file(url, str(id))
    print("file downloaded " + url)
    raw_text = digitize_file('/content/' + str(id))
    try:
      raw_text[0] = raw_text[0]['content']
    except:
      pass
    raw_text = ''.join(raw_text)
    print("file parsed")
    print(raw_text)
    app_search.index_documents(
        engine_name=engine_name_full_text,
        body=[{
              "id": id,
              "title": title,
              "abstract": abstract,
              "authors": authors,
              "publication_date": publication_date,
              "field": field,
              "keywords": keywords,
              "type": doc_type,
              "publisher": publisher,
              "url": url,
              "identifier_uri": identifier_uri,
              "language": language,
              "body": raw_text
        }]
    )

In [54]:
yearly_results = app_search.search(
        engine_name=engine_name,
        body={
            "query": "",
            "page": {
                "size": 10,
            },
            "filters" : {
                "all":[
              {"publication_date": 2020},
              {"publisher": "Université Mohamed Boudiaf - M'sila"}]
            }
        }
    )
results = yearly_results['results']
print(len(results))

10


In [110]:
index_fulltext(results[0])

Elimination de la pollution industrielle de textile par électrocoagulation
file downloaded http://dspace.univ-msila.dz:8080/xmlui/bitstream/handle/123456789/20693/1011.pdf?sequence=1&isAllowed=y
going for Plumber
file parsed
                                                   
REPUBLIQUE ALGERIENNE DEMOCRATIQUE ET POPULAIRE 
MINISTERE DE L’ENSEIGNEMENT SUPERIEUR ET DE LA RECHERCHE SCIENTIFIQUE 
UNIVERSITE MOHAMED BOUDIAF - M’SILA 
 
  DOMAINE : SCIENCES ET TECHNOLOGIES 
FACULTE : TECHNOLOGIE   
 
FILIERE : HYDRAULIQUE 
  DEPARTEMENT : HYDRAULIQUE 
  OPTION : RESSOURCES HYDRAULIQUES  
N° :………………………………………..                                                                                     
   
 
 
 
                                                                         
 
 
Mémoire présenté pour l’obtention 
du diplôme de Master Académique 
 
 
Par: Hadidane Ramzi
 
et 
Boussalem Hocine 
               
 
 
 
Intitulé 
 
 
Elimination de la pollution industrielle de textile  
 
  par é

In [111]:
result = app_search.search(
        engine_name=engine_name,
        body={
            "query": "",
            "page": {
                "size": 10,
            },
            "filters" : {
                "all":[
              {"id": '7859947258069749'},
              ]
            }
        }
    )

In [112]:
result['meta']['page']['total_results'] > 0

True

In [113]:
result['results'][0]

{'_meta': {'engine': 'theses-dz', 'id': '7859947258069749', 'score': 1.0},
 'abstract': {'raw': "Résumé\r\nL’objectif de ce présent travail est d’évaluer les performances du procédé d'électrocoagulation\r\n(EC) en continu pour l’élimination du colorant Rouge Nylosan présent dans les effluents des\r\nindustries du textile.\r\nL’effet de différents paramètres de fonctionnement susceptibles d’influer le procédé, tels que, la\r\ndensité de courant, le temps de traitement, le mode de connexion des électrodes (bipolaire BP,\r\nmonopolaire en parallèle MP-P et monopolaire en série MP-S), le nombre des électrodes, et le\r\nrendement faradique a été étudié.\r\nLes résultats trouvés ont montré que l’EC a bien fonctionné pour tous les modes de connexion\r\nétudiés, toutefois, la connexion monopolaire parallèle (MP-P) à 2 électrodes et à 4 électrodes est\r\nla plus efficace sur tous les plans y compris celui de la consommation énergétique, et le\r\nrendement faradique moyen est supérieur à l’unité